# Introduction {-}

In this tutorial, we will learn to how to implement a logistic mixed-effects model.

**Preparation and session set up**

Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead and ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# install packages
install.packages("here")
install.packages("dplyr")
install.packages("ggplot2")
install.packages("glmulti")
install.packages("sjPlot")
install.packages("report")
install.packages("lme4")
install.packages("rms")
install.packages("car")


Now that we have installed the packages, we activate them as shown below.



In [ ]:
# activate packages
library(here)
library(dplyr)
library(ggplot2)
library(glmulti)
library(sjPlot)
library(report)
library(lme4)
library(rms)
library(car)


#  Tutorial Activity {-}

Go into groups - each group and help each other to bring the data into the correct format, visualize the data and perform the logistic mixed-effects regression.

# Task 1

In this example, we want to see what factors impact the use of *eh* in New Zealand English.

Load the data set `week12d1.xlsx`. Visualize the data and perform a logistic mixed-effects regression to determine what factors impact if a speech unit ends with *eh*. 


In [ ]:
dat1 <- readxl::read_excel(here::here("data", "week12d1.xlsx")) 
# inspect
head(dat1)


Prepare data



In [ ]:
dat1 <- dat1 %>% 
  dplyr::mutate_if(is.character, factor)
# inspect
head(dat1)


Check nestedness



In [ ]:
table(dat1$Speaker)



In [ ]:
table(dat1$Gender, dat1$EH)



Visualize data



In [ ]:
dat1  %>%
  ggplot(aes(EH, fill = EH)) +
  geom_bar(stat = "count") + 
  facet_grid(Gender ~ Age)


Set options



In [ ]:
# set contrasts
options(contrasts  =c("contr.treatment", "contr.poly"))
# extract distribution summaries for all potential variables
blrdata.dist <- datadist(dat1)
# store distribution summaries for all potential variables
options(datadist = "blrdata.dist")


Model fitting

Manual


In [ ]:
m0 <- glmer(EH ~ (1|Speaker) + 1, data = dat1, family = binomial)



Automated



In [ ]:
# wrapper function for linear mixed-models
glmer.glmulti <- function(formula,data, random="",...){
  glmer(paste(deparse(formula),random), family = binomial, data=data, ...)
}
# define formular
form_glmulti = as.formula(paste("EH ~ Gender + Age + Ethnicity"))


# multi selection for glmer
mfit <- glmulti(form_glmulti,random="+(1 | Speaker)", 
                data = dat1, method = "h", fitfunc = glmer.glmulti,
                crit = "aic")
# extract best models
top <- weightable(mfit)
top <- top[1:10,]
# inspect top 10 models
top


Define final minimal adequate model

**WARNING!**: We should not be using this model due to substantive multi-collinearity (isSingular)


In [ ]:
m1 <- glmer(EH ~ (1|Speaker) + Gender + Age + Ethnicity, family = "binomial", data = dat1)
# inspect results
summary(m1)


Diagnostics

Multicolliniarity


In [ ]:
car::vif(m1)



All good: the vif values are smaller than 5!

Outliers?


In [ ]:
plot(m1)



Effects



In [ ]:
sjPlot::plot_model(m1, type = "pred", terms = c("Gender", "Age", "Ethnicity"))



Summarize



In [ ]:
sjPlot::tab_model(m1)



Report



In [ ]:
report::report(m1)



# Outro



In [ ]:
sessionInfo()

